In [1]:
# Import Libraries
from googleapiclient.discovery import build
import pandas as pd
from ipywidgets import FileUpload
import json

In [2]:
# Set Up the YouTube API
API_KEY = "YOUR API KEY HERE" 
youtube = build('youtube', 'v3', developerKey=API_KEY)

In [3]:
import ipywidgets as widgets
from IPython.display import display

# Create File Upload Widget
upload = FileUpload(accept='.csv', multiple=False)

# Create a Button to Confirm Upload
button = widgets.Button(description="Confirm Upload")

# Display Widgets
display(upload, button)

# Define a Callback for the Button
def on_button_click(b):
    if upload.value:
        print("File uploaded successfully!")
    else:
        print("No file uploaded. Please upload a file.")

# Link Button to Callback
button.on_click(on_button_click)



FileUpload(value=(), accept='.csv', description='Upload')

Button(description='Confirm Upload', style=ButtonStyle())

In [4]:
# Process Uploaded File
import pandas as pd
import io

# Process Uploaded File
if upload.value:
    uploaded_file = upload.value[0]  # Access the first file in the tuple
    content = uploaded_file['content'].tobytes().decode('utf-8')  # Convert memory to bytes, then decode
    search_terms = pd.read_csv(io.StringIO(content)).iloc[:, 0].tolist()  # Extract terms from the first column
    print(f"Search terms loaded: {search_terms}")
else:
    print("Please upload a CSV file.")



Search terms loaded: ['weatherproof decks', 'types of decks', 'deck diy ', 'composite deck']


In [5]:
import json
from pathlib import Path

# File to store cached category mapping
CATEGORY_CACHE_FILE = Path("category_mapping.json")

def get_video_categories():
    """
    Fetch video category mappings (ID to name) and cache them for efficiency.
    """
    # Check if the cache file exists
    if CATEGORY_CACHE_FILE.exists():
        with open(CATEGORY_CACHE_FILE, "r") as f:
            print("Using cached category mapping.")
            return json.load(f)  # Load the cached mapping

    # Fetch category mapping from the YouTube API
    response = youtube.videoCategories().list(
        part="snippet",
        regionCode="US"  # Specify the region for categories
    ).execute()

    # Create a mapping of category ID to category name
    category_mapping = {
        item['id']: item['snippet']['title']
        for item in response['items']
    }

    # Cache the mapping for future use
    with open(CATEGORY_CACHE_FILE, "w") as f:
        json.dump(category_mapping, f)
        print("Category mapping cached.")

    return category_mapping


def fetch_youtube_data(search_term, max_results=10):
    """
    Fetch video data for a given search term, including mapped category names.
    """
    # Get the cached or fresh category mapping
    category_mapping = get_video_categories()

    # Make a search request to fetch video IDs and snippet
    search_response = youtube.search().list(
        q=search_term,
        part="snippet",
        type="video",
        maxResults=max_results
    ).execute()

    # Extract video IDs from the search response
    video_ids = [item['id']['videoId'] for item in search_response['items']]

    # Fetch additional details for the videos
    stats_response = youtube.videos().list(
        part="snippet,statistics,contentDetails",
        id=",".join(video_ids)  # Join video IDs into a comma-separated string
    ).execute()

    video_data = []

    # Process each video
    for idx, item in enumerate(stats_response['items'], start=1):
        try:
            stats = item.get('statistics', {})
            content_details = item.get('contentDetails', {})
            category_id = item['snippet'].get('categoryId', 'N/A')

            # Combine video details with statistics
            video_details = {
                'Search Term': search_term,
                'Position': idx,
                'Video Title': item['snippet']['title'],
                'Channel Name': item['snippet']['channelTitle'],
                'Published Date': item['snippet']['publishedAt'],
                'Description': item['snippet']['description'],
                'Video URL': f"https://www.youtube.com/watch?v={item['id']}",
                'View Count': stats.get('viewCount', 'N/A'),
                'Like Count': stats.get('likeCount', 'N/A'),
                'Comments Count': stats.get('commentCount', 'N/A'),
                'Duration': content_details.get('duration', 'N/A'),
                'Thumbnail URL': item['snippet']['thumbnails']['default']['url'],
                'Live Status': item['snippet'].get('liveBroadcastContent', 'N/A'),
                'Category Name': category_mapping.get(category_id, 'Unknown')  # Map ID to name
            }
            video_data.append(video_details)

        except KeyError as e:
            print(f"KeyError for video: {e}, skipping...")
        except Exception as e:
            print(f"Error processing video: {e}, skipping...")

    return video_data



In [6]:
# Fetch Data for All Search Terms
all_video_data = []

try:
    for term in search_terms:
        print(f"Fetching data for search term: {term}")
        video_data = fetch_youtube_data(term, max_results=10)
        all_video_data.extend(video_data)
except Exception as e:
    print(f"An error occurred: {e}")
    raise  # Stop execution on error

Fetching data for search term: weatherproof decks
Category mapping cached.
Fetching data for search term: types of decks
Using cached category mapping.
Fetching data for search term: deck diy 
Using cached category mapping.
Fetching data for search term: composite deck
Using cached category mapping.


In [7]:
# Convert to DataFrame and Save to CSV
df = pd.DataFrame(all_video_data)
output_file = "youtube_data_with_positions.csv"
df.to_csv(output_file, index=False)
print(f"Data saved to {output_file}")

Data saved to youtube_data_with_positions.csv
